# Modeling and Simulation in Python

Chapter 10: Vectors

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# If you want the figures to appear in the notebook, 
# and you want to interact with them, use
# %matplotlib notebook

# If you want the figures to appear in the notebook, 
# and you don't want to interact with them, use
# %matplotlib inline

# If you want the figures to appear in separate windows, use
# %matplotlib qt5

# to switch from one to another, you have to select Kernel->Restart

%matplotlib notebook

from modsim import *

### Vectors

A `Vector` object represents a vector quantity.  In the context of mechanics, vector quantities include position, velocity, acceleration, and force, all of which might be in 2D or 3D.

You can define a `Vector` object without units, but if it represents a physical quantity, you will often want to attach units to it.

I'll start by grabbing the units we'll need.

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram

Here's a two dimensional `Vector` in meters.

In [3]:
A = Vector(3, 4) * m

We can access the elements by name.

In [4]:
A.x

<Quantity(3.0, 'meter')>

In [5]:
A.y

<Quantity(4.0, 'meter')>

The magnitude is the length of the vector.

In [6]:
A.mag

<Quantity(5.0, 'meter')>

The angle is the number of radians between the vector and the positive x axis.

In [7]:
A.angle

<Quantity(0.9272952180016122, 'radian')>

If we make another `Vector` with the same units,

In [8]:
B = Vector(1, 2) * m

We can add `Vector` objects like this

In [9]:
A + B

<Quantity([ 4.  6.], 'meter')>

And subtract like this:

In [10]:
A - B

<Quantity([ 2.  2.], 'meter')>

We can compute the Euclidean distance between two Vectors.

In [11]:
A.dist(B)

<Quantity(2.8284271247461903, 'meter')>

And the difference in angle

In [12]:
A.diff_angle(B)

<Quantity(-0.17985349979247822, 'radian')>

If we are given the magnitude and angle of a vector, what we have is the representation of the vector in polar coordinates.

In [13]:
mag = A.mag
angle = A.angle

We can use `pol2cart` to convert from polar to Cartesian coordinates, and then use the Cartesian coordinates to make a `Vector` object.

In this example, the `Vector` we get should have the same components as `A`.

In [14]:
x, y = pol2cart(angle, mag)
Vector(x, y)

<Quantity([ 3.  4.], 'meter')>

**Exercise:** Create a `Vector` named `a_grav` that represents acceleration due to gravity, with x component 0 and y component $-9.8$ meters / second$^2$.

In [15]:
# Solution

a_grav = Vector(0, -9.8 * m / s**2)
a_grav

<Quantity([ 0.  -9.8], 'meter / second ** 2')>

### Degrees and radians

Pint provides units to represent degree and radians.

In [16]:
degree = UNITS.degree
radian = UNITS.radian

If you have an angle in degrees,

In [17]:
angle = 45 * degree
angle

<Quantity(45, 'degree')>

You can convert to radians.

In [18]:
angle_rad = angle.to(radian)
angle_rad

<Quantity(0.7853981633974483, 'radian')>

If it's already in radians, `to` does the right thing.

In [19]:
angle_rad.to(radian)

<Quantity(0.7853981633974483, 'radian')>

You can also convert from radians to degrees.

In [20]:
angle_rad.to(degree)

<Quantity(45.0, 'degree')>

As an alterative, you can use `np.deg2rad`, which works with Pint quantities, but it also works with simple numbers and NumPy arrays:

In [21]:
np.deg2rad(angle)

<Quantity(0.7853981633974483, 'radian')>

**Exercise:** Create a `Vector` named `a_force` that represents acceleration due to a force of 0.5 Newton applied to an object with mass 0.3 kilograms, in a direction 45 degrees up from the positive x-axis.

Add `a_force` to `a_drag` from the previous exercise.  If that addition succeeds, that means that the units are compatible.  Confirm that the total acceleration seems to make sense.

In [22]:
# Solution

N = UNITS.newton
mag = 0.5 * N
angle = 45 * degree
theta = angle.to(radian)
x, y = pol2cart(theta, mag)
force = Vector(x, y)
mass = 0.3 * kg
a_force = force / mass
a_force

<Quantity([ 1.1785113  1.1785113], 'newton / kilogram')>

In [23]:
# Solution

a_force + a_grav

<Quantity([ 1.1785113 -8.6214887], 'newton / kilogram')>

### Thinking in Vectors

When you work on mechanics problems, a gravity-like force pulls you toward writing equations and code in terms of components rather than vectors.  If you resist this force, you will produce equations and code that are easier to read and more likely to be correct.  The best way to resist this force is to "think in vectors".

1.  Whenever possible, express computations in terms of Vector objects rather than components.

2.  If you are given components at the beginning of a computation, assemble them into Vectors as soon as possible.  If you are required to produce components at the end of a computation, delay extracting them as long as possible.

3. Whenever you perform operations on vector quantities, think about how to express them as `Vector` operations.  Avoid unpacking the components if you can.

In Chapter 10, you will see a few examples of Vector thinking.  For example, here's the slope function that computes acceleration due to gravity and drag.

In [24]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)
    
    a_grav = Vector(0, -g)

    v = Vector(vx, vy)
    
    f_drag = -rho * v.mag * v * C_d * area / 2
    a_drag = f_drag / mass
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y

A few things to comment on here:

1.  `The System` object provides the magnitude of acceleration due to gravity, but not the direction.  Putting this magnitude into a Vector makes an implicit part of the program explicit.

2.  In order to work with `odeint`, the state variables have to be scalars, not Vectors.  So I assemble `vx` and `vy` into a `Vector` as soon as possible.

3.  To represent `f_drag`, we want to create a `Vector` that has the magnitude given by the drag equation and a direction opposite the direction of `v`.  There are two ways to think about that, which I'll explain below.

4.  Because `a_grav` and `a_drag` are vectors that encode their direction, we can add them up without having to think about their signs.  Don't think about subtracting gravity from other forces, because it points down!  Just think about adding up vectors.

5.  It is a common error to add forces to accelerations.  By attaching units to all `Vector` object, we catch that error right away.

In [25]:
condition = Condition(x = 0 * m, 
                      y = 1 * m,
                      g = 9.8 * m/s**2,
                      mass = 145e-3 * kg,
                      diameter = 73e-3 * m,
                      rho = 1.2 * kg/m**3,
                      C_d = 0.3,
                      angle = 45 * degree,
                      velocity = 40 * m / s,
                      duration = 5.1 * s)

In [26]:
def make_system(condition):
    """Make a system object.
    
    condition: Condition object with angle, velocity, x, y,
               diameter, duration, g, mass, rho, and C_d
               
    returns: System object
    """
    unpack(condition)
    
    # convert angle to degrees
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, velocity)
    
    # make the initial state
    init = State(x=x, y=y, vx=vx, vy=vy)
    
    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    # compute timestamps
    ts = linspace(0, duration, 101)
    
    return System(init=init, g=g, mass=mass, 
                  area=area, rho=rho, C_d=C_d, ts=ts)

In [27]:
system = make_system(condition)
system

,value
init,x 0 meter y ...
g,9.8 meter / second ** 2
mass,0.145 kilogram
area,0.004185386812745002 meter ** 2
rho,1.2 kilogram / meter ** 3
C_d,0.3
ts,"[0.0 second, 0.051 second, 0.102 second, 0.153..."


In [28]:
slope_func(system.init, 0, system)

(<Quantity(28.284271247461902, 'meter / second')>,
 <Quantity(28.2842712474619, 'meter / second')>,
 <Quantity(-5.878209892331404, 'meter / second ** 2')>,
 <Quantity(-15.678209892331404, 'meter / second ** 2')>)

In [29]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)
    
    a_grav = Vector(0, -g)

    v = Vector(vx, vy)
    
    f_drag_mag = -rho * v.mag**2 * C_d * area / 2    # scalar
    f_drag_dir = v / v.mag                           # vector
    f_drag = f_drag_mag * f_drag_dir
    
    a_drag = f_drag / mass
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y

In [30]:
slope_func(system.init, 0, system)

(<Quantity(28.284271247461902, 'meter / second')>,
 <Quantity(28.2842712474619, 'meter / second')>,
 <Quantity(-5.878209892331404, 'meter / second ** 2')>,
 <Quantity(-15.678209892331406, 'meter / second ** 2')>)

In [31]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)
    
    a_grav = Vector(0, -g)

    v = Vector(vx, vy)
    
    f_drag_mag = -rho * v.mag**2 * C_d * area / 2    # scalar
    f_drag_dir = v.hat()                             # vector
    f_drag = f_drag_mag * f_drag_dir
    
    a_drag = f_drag / mass
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y

In [32]:
slope_func(system.init, 0, system)

(<Quantity(28.284271247461902, 'meter / second')>,
 <Quantity(28.2842712474619, 'meter / second')>,
 <Quantity(-5.878209892331404, 'meter / second ** 2')>,
 <Quantity(-15.678209892331406, 'meter / second ** 2')>)